# Nicholas W.: Forecasting Consensus Expectations of Nonfarm Payrolls (NFP)

# Miscellaneous experiments

In [2]:
import os
import warnings
import math
import json
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

from tqdm.auto import tqdm
from typing import List, Tuple, Dict, Any
from scipy import stats, special
from scipy.optimize import brentq, minimize
from scipy.stats import t as student_t, norm, binomtest, jarque_bera
from sklearn.mixture import GaussianMixture
from collections import defaultdict
from itertools import product
from arch.univariate import ConstantMean, GARCH, StudentsT
from arch.univariate.base import ConvergenceWarning
from IPython.display import display, Markdown
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf
from pandas.tseries.offsets import MonthBegin

warnings.filterwarnings("ignore")

In [3]:
OUT_DIR = "../out"        
DF_FILE       = "nfp_df.parquet"
DF_FULL_FILE  = "nfp_df_full.parquet"

df       = pd.read_parquet(os.path.join(OUT_DIR, DF_FILE),      engine="pyarrow")
df_full  = pd.read_parquet(os.path.join(OUT_DIR, DF_FULL_FILE), engine="pyarrow")

print("df shape     :", df.shape)
print("df_full shape:", df_full.shape)

df shape     : (17046, 10)
df_full shape: (19375, 10)


## E1: Optimal rolling window length for uncertainty quantification and confidence intervals built around uncertainty in the cross-section

In this section, we test empirically whether accurate confidence intervals can be built using solely cross-sectional spread (standard deviation of forecasts). We also test different rolling window lengths for student-t confidence intervals.

Conclusion: Confidence intervals **cannot** be built solely from cross-sectional forecast spread and we find that a **36-month** rolling window length provides the tightest empirical coverage relative to nominal. 

In [ ]:
LEVELS  = [0.50, 0.60, 0.70, 0.80, 0.90, 0.95]
TS_WINS = [12, 24, 36, 60, 120]          # months
MIN_XS  = 5                          # min forecasts in cross-section
PANELS  = {"COVID": df, "Full": df_full}

def _mean_abs_gap(empirical_vec, levels=LEVELS):
    """Mean-absolute gap between empirical and nominal coverages."""
    return float(np.abs(np.array(empirical_vec) - np.array(levels)).mean())

def in_band(center, sig, nu, level, actual):
    half = st.t.ppf(1 - (1-level)/2, df=nu) * sig
    return int(center - half <= actual <= center + half)

rows = []

for panel_name, panel in PANELS.items():

    # cross-section 
    xs_hits = {L: 0 for L in LEVELS};  xs_tot = 0

    for date, grp in tqdm(panel.groupby("release_date"), desc=f"{panel_name} XS"):
        sample = grp["forecast"].dropna().values
        act    = grp["actual"].iloc[0]
        if len(sample) < MIN_XS or np.isnan(act):
            continue
        nu, loc, sig = st.t.fit(sample)           # μ̂ = loc
        for L in LEVELS:
            xs_hits[L] += in_band(loc, sig, nu, L, act)
        xs_tot += 1
    
    emp_vec = [xs_hits[L] / xs_tot for L in LEVELS]   # six empirical coverages
    mag     = _mean_abs_gap(emp_vec) 

    for L, e in zip(LEVELS, emp_vec):
        rows.append({"Method": f"{panel_name}-XS-t",
                     "Nominal": L,
                     "Empirical": e,
                     "MAG": mag})

    # time-series
    ts_hits = {w:{L:0 for L in LEVELS} for w in TS_WINS}
    ts_tot  = {w:0 for w in TS_WINS}

    panel_sorted = panel.sort_values("release_date")
    err_series = (panel_sorted.groupby("release_date")
                                .apply(lambda g: g["median_forecast"].iloc[0] - g["actual"].iloc[0])
                                .dropna())
    dates = err_series.index.to_list()

    for win in TS_WINS:
        for i in tqdm(range(win, len(dates)), desc=f"{panel_name} TS {win}m", leave=False):
            train_errs = err_series.iloc[i-win:i].values
            if train_errs.size < win:
                continue
            nu, mu, sig = st.t.fit(train_errs)

            cur_date   = dates[i]
            slc        = panel_sorted[panel_sorted["release_date"] == cur_date]
            point_med  = slc["median_forecast"].iloc[0]   # centre at median + μ̂
            actual_val = slc["actual"].iloc[0]
            if np.isnan(actual_val):
                continue

            center_ts  = point_med + mu                   # overall centre
            for L in LEVELS:
                ts_hits[win][L] += in_band(center_ts, sig, nu, L, actual_val)
            ts_tot[win] += 1
            
        emp_vec_win = [ts_hits[win][L] / ts_tot[win] for L in LEVELS]
        mag_win     = _mean_abs_gap(emp_vec_win)

        for L, e in zip(LEVELS, emp_vec_win):
            rows.append({"Method": f"{panel_name}-TS-t_{win}m",
                         "Nominal": L,
                         "Empirical": e,
                         "MAG": mag})

df = pd.DataFrame(rows)

coverage_df = (df
               .pivot(index="Method", columns="Nominal", values="Empirical")
               .sort_index())

coverage_df["MAG"] = (df
                      .drop_duplicates("Method")
                      .set_index("Method")["MAG"]
                      .reindex(coverage_df.index))

print("\nEmpirical vs nominal coverage (XS/TS-t centred at μ̂)")
print(coverage_df.to_string(float_format=lambda x: f"{x:0.3f}"))


COVID XS:   0%|          | 0/231 [00:00<?, ?it/s]

TypeError: cannot unpack non-iterable float object